In [93]:
from genet.core import Network
import networkx as nx
from tqdm import tqdm
from lxml import etree as et
from lxml.etree import Element, SubElement

In [2]:
n = Network()

In [3]:
pt_net_path = '/Users/alex.andrey/Documents/city_modelling/ireland/network.xml'
# pt_sch_path = '/Users/kasia.kozlowska/PycharmProjects/ABM/genet/example_data/pt2matsim_network/schedule.xml'

In [4]:
n.read_matsim_network(pt_net_path, epsg='epsg:27700')

/Users/alex.andrey/Library/Python/3.6/lib/python/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/alex.andrey/Library/Python/3.6/lib/python/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/Users/alex.andrey/Library/Python/3.6/lib/python/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the pr

In [5]:
nx.info(n.graph)

'Name: \nType: MultiDiGraph\nNumber of nodes: 382256\nNumber of edges: 857068\nAverage in degree:   2.2421\nAverage out degree:   2.2421'

### Load the OSM way ids of the toll links

In [14]:
osm_ids_path = '/Users/alex.andrey/Documents/city_modelling/ireland/osm_toll_way_ids'

In [73]:
with open(osm_ids_path,'r') as f:
    osm_way_ids = [line.rstrip('\n') for line in f]

In [90]:
osm_way_ids = list(set(osm_way_ids)) # remove duplicates

### Extract all edges from network.xml  whose `['attributes']['osm:way:id']['text']` is present in `osm_way_ids`

In [78]:
network_toll_ids = []
edge_osm_ids = []

for edge_dict in n.edges():
    for key in n.edges(edge_dict):

        if 'attributes' in n.edges(edge_dict)[key].keys():

            edge_osm_id = n.edges(edge_dict)[key]['attributes']['osm:way:id']['text']        

            if edge_osm_id in osm_way_ids:
                network_toll_ids.append(
                    n.edges(edge_dict)[key]['id']
                )

                edge_osm_ids.append(edge_osm_id)


    edge_osm_ids = list(set(edge_osm_ids))
    if edge_osm_ids == osm_way_ids:
        break


224824835it [00:27, 8171884.73it/s]    


In [85]:
print(len(network_toll_ids), ' network ids identified as matching OSM toll ids')

614  network ids identified as matching OSM toll ids


In [91]:
print(len(osm_way_ids)-len(edge_osm_ids),' OSM toll ids not found in network.xml')

2  OSM toll ids not found in network.xml


In [92]:
missing_osm_ways = [item for item in osm_way_ids if item not in edge_osm_ids]
missing_osm_ways

['760065900', '760065899']

User can investigate the missing links using [OpenStreetMap web interface](https://www.openstreetmap.org/way/760065899#map=13/53.2934/-8.3791)

## Build XML

for now, build Road Pricing scheme [Cordon Count](https://www.matsim.org/apidocs/core/0.4.0/org/matsim/roadpricing/package-summary.html)

In [ ]:
out_path = '/Users/alex.andrey/Documents/city_modelling/ireland/roadpricing-file.xml'

In [94]:
def write_xml(root, path):
        """
        Write config to given xml path.
        :param path: pathlib.Path
        :return: None
        """
        # if isinstance(path, str):
        #     path = Path(path)
        # root = self.build_xml()
        tree = et.tostring(root,
                           pretty_print=True,
                           xml_declaration=False,
                           encoding='UTF-8')
        with open(path, 'wb') as file:
            file.write(b'<?xml version="1.0" ?>\n')
            file.write(b'<!DOCTYPE roadpricing SYSTEM "http://matsim.org/files/dtd/roadpricing_v1.dtd">\n')
            file.write(tree)

In [103]:
# creat ETree root
roadpricing = Element("roadpricing", type="cordon", name="cordon-toll")
# <description
description = SubElement(roadpricing, "description")
description.text = "A simple cordon toll scheme for the Ireland network"

In [104]:
links = SubElement(roadpricing, "links")

for link_id in network_toll_ids: 
    SubElement(links, "link", id=link_id)

In [105]:
# apply same cost to all links, regardless of vehicle type
SubElement(roadpricing, "cost", start_time="00:00", end_time="23:59", amount="2.00")

<Element cost at 0x22dcfbfc8>

In [106]:
write_xml(roadpricing, out_path)